In [1]:
from pymongo import MongoClient

# single machine setup (mongo is running here localy)
MY_DATABASE_LINK = 'mongodb://192.168.1.191:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

class DatabaseConnect:
    DB_LINK = MY_DATABASE_LINK

    class Base:
        client = None
        @classmethod
        def connect(cls, databaseName):
            cls.client = MongoClient(DatabaseConnect.DB_LINK)
            # Access the database
            return cls.client[databaseName]

        @classmethod
        def close(cls):
            if cls.client:
                cls.client.close()
                cls.client = None

        @classmethod
        def getCollection(cls, collectionName):
            return cls.client[collectionName]


    class mini_database(Base):
        @classmethod
        def projects(cls):
            #print(cls.connect)
            
            return cls.connect('mini_database')['projects']
        @classmethod
        def users(cls):
            return cls.connect('mini_database')['users']

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['192.168.43.146:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [2]:
import numpy as np

In [3]:
def getProjectsStars(projectsNum : int) -> list:
    count = projectsNum
    cursor = projectsCollection.find()
    stars = []

    for proj in cursor:
        if count <= 0: break
        stars.append(proj["stars"])
        count -= 1

    return np.array(stars)

def getProjectsForPeriods(stars : list, periodSize = 50):
    class Period(list):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.projectsAmount = 0

        def includes(self, n):
            return (self[0] <= n) and (self[1] >= n)


    maxVal = max(stars)
    print(maxVal)
    # construct periods:
    periods = [Period((i, i + periodSize)) for i in range(0, maxVal + periodSize, periodSize)]
    if maxVal % periodSize: periods += [Period((maxVal - (maxVal % periodSize), maxVal))]

    print(len(periods))
    projectsPerPeriod = [0] * len(periods)
    projectsCursor = projectsCollection.find()

    for projectStars in stars:
        for index, period in enumerate(periods):
            if period.includes(projectStars):
                period.projectsAmount += 1
                #projectsPerPeriod[index] += 1

    return {"periods" : periods, "projects_amount" : projectsPerPeriod}

In [4]:
starsArray = getProjectsStars(10000)

maximum = max(starsArray)
maximum

ServerSelectionTimeoutError: 192.168.43.146:27020: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67d8a4cde9d87a8575d9afe9, topology_type: Unknown, servers: [<ServerDescription ('192.168.43.146', 27020) server_type: Unknown, rtt: None, error=NetworkTimeout('192.168.43.146:27020: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
def percentile_threshold(likes, percentile=50):
    return np.percentile(likes, percentile)

threshold = percentile_threshold(starsArray, 70)  # Keep top 30% posts

print("Threshold:", threshold)
print(f"Original size: {len(starsArray)}")
print(len(starsArray[starsArray > threshold]))

In [ ]:
nonZero = np.array([p.projectsAmount for p in projectsForPeriods])
nonZero = nonZero[nonZero > 0]
calculateOptimal(nonZero, 2)

In [ ]:
nonZero

In [ ]:
from copy import deepcopy

def projectDataIsValid(projectData : dict):
    # will check if project data can actually be used in the training (can specify other conditions)
    return projectData and len(projectData["description"]) > 0

def getProjectsDataForUsers(usersNumber = float("inf")) -> dict[str, list]:
    # will return a dictionary, where keys are users ids and values are lists of projects ids, each user has contributed to
    i = 0
    count = usersNumber
    cursor = usersCollection.find()
    data = {}

    for user in cursor:
        if count <= 0: break
        print(f"Scanning user: {i}")
        projectsIDList = user["projects"]

        projects = []

        for proj_id in projectsIDList:
            projectData = projectsCollection.find_one({"id" : proj_id}, {"_id" : False})

            if projectDataIsValid(projectData):
                projects.append(projectData)
        
        if len(projects):
            data[user["id"]] = deepcopy(projects)
            count -= 1

        i += 1

    return data

getProjectsDataForUsers(2)

In [ ]:
def textPreprocessing(text):
    import nltk
    from nltk.corpus import stopwords
    #from nltk.stem import WordNetLemmatizer
    import string

    # Initialize tools
    stop_words = set(stopwords.words("english"))
    #lemmatizer = WordNetLemmatizer()

    
    result = text.

    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize into words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Remove stopwords & lemmatize
    return tokens


def projectsDataPreprocessing(projects : np.array[dict]) -> np.array[{"text" : str, "tags" : list}]:
    # will take in an array of projects and prepare it to be consumed by the model
    # takes: array of projects (as dictionaries); returns: text data and tags for every project in array
    for proj in projects:
        text = ". ".join([proj["name"], proj["description"]])
        tags = [proj["language"], *proj["topics"]] if proj["language"] else proj["topics"]

        

In [21]:
import re

def processWord(camelCase):
    
    template = "[a-z][A-Z]"
    text = "jhgfYjhgbvfTfvbnNsderftghbvN"
    result = ""
    n = 0
    spans = []

    while found := re.search(template, text[n:]):
        spans.append((found.span()[0] + n, found.span()[1] + n))
        n += found.span()[1]
    
    print([text[span[0]:span[1]] for span in spans])

    for i, span in enumerate(spans):
        if i == -1: break
        start = span[0] + i
        end = span[1] + i - 1
        print(start, end)
        text = text[:start] + f"{text[start]} {text[end].lower()}" + text[end + 1:]

    return text

processWord("sdfDfg")

['fY', 'fT', 'nN', 'vN']
3 4
11 12
17 18
29 30


'jhgf yjhgbvf tfvbn nsderftghbv n'

In [ ]:
"""
#def camelCaseProcess(text):
template = "[a-zA-Z]+[A-Z]+[a-z]*[A-Z]*[a-z]*[A-Z]*[a-z]*[A-Z]*[a-z]*"
text = "qwerfgvfdE cdfgWertghn KHGVBNMK HgvgEvc jhgfYjhgbvfTfvbnNsderftghbv"

#found = re.findall(template, text)
#for item in found:
#    template = template.replace()

#print(found)
"""

(0, 'q')